In [1]:
pip install rdflib

     |████████████████████████████████| 235kB 8.2MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 


In [3]:
import rdflib
from rdflib import URIRef, BNode, Literal, Graph
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
from rdflib import Namespace
import csv
from csv import DictReader

In [5]:
path = "query (7).csv" #original file
namespace = "http://example.org/" #needs to be the same namespace of the construct query
filename = "aboxexhibition.ttl" #newfile
prefix = "ex"

In [8]:
def csvtoturtle(fname, prefix, filepath, nspace):
  ont = Namespace(nspace)
  wd = Namespace("http://www.wikidata.org/entity/")
  wdt = Namespace("http://www.wikidata.org/prop/direct/")
  g = Graph() 
  g.bind(prefix, ont) 
  g.bind("rdf",  RDF)
  g.bind("wd", wd)
  g.bind("wdt", wdt)
  with open(path, "r") as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
      s = row["subject"]
      p = row["predicate"]
      o = row["object"]
      if "http" in o:
        g.add((URIRef(s), URIRef(p), URIRef(o)))
      else:
        g.add((URIRef(s), URIRef(p), Literal(o)))
  print(g.serialize(format="turtle").decode("utf-8"))
  g.serialize(destination = fname, format="turtle")


In [9]:
csvtoturtle(filename, prefix, path, namespace)

@prefix ex: <http://example.org/> .
@prefix wd: <http://www.wikidata.org/entity/> .

wd:Q12418 ex:something ex:Exhibition1 ;
    ex:somethingelse "Mona Lisa" .

wd:Q3211030 ex:something ex:Exhibition1 ;
    ex:somethingelse "Birth of Venus" .

ex:Exhibition1 a ex:Exhibition .




# Add more triples to an existing graph


In [24]:
moretriples = [("http://www.wikidata.org/entity/Q12418", "doesNotBelongTo", "Exhibition2"),
               ("Exhibition1", "title", "Literal:The ancient Roman Empire Exhibition")]

In [25]:
def addmoretriples(nspace, prefix, gr, additionaltriples):
  ont = Namespace(nspace)
  wd = Namespace("http://www.wikidata.org/entity/")
  wdt = Namespace("http://www.wikidata.org/prop/direct/")
  newg = rdflib.Graph() 
  newg.parse(gr, format="ttl")
  newg.bind(prefix, ont) 
  newg.bind("rdf",  RDF)
  newg.bind("wd", wd)
  newg.bind("wdt", wdt)
  for triple in additionaltriples:
    if "http" in triple[0]:
      s = URIRef(triple[0])
    else:
      s = URIRef(nspace+triple[0])
    if "http" in triple[1]:
      p = URIRef(triple[1])
    elif triple[1] == "type":
      p = URIRef(RDF.type)
    else:
      p = URIRef(nspace+triple[1])
    if "http" in triple[2]:
      o = URIRef(triple[2])
    elif "Literal:" in triple[2]:
      o = Literal(triple[2].split("Literal:")[1])
    else:
      o = URIRef(nspace+triple[2])
    newg.add((s,p,o))
  print(newg.serialize(format="turtle").decode("utf-8"))
  newg.serialize(destination = "new"+gr, format="turtle")


In [26]:
addmoretriples("http://example.org/", "ex", "graph.ttl", moretriples)

@prefix ex: <http://example.org/> .
@prefix wd: <http://www.wikidata.org/entity/> .

wd:Q12418 ex:doesNotBelongTo ex:Exhibition2 ;
    ex:something ex:Exhibition1 ;
    ex:somethingelse "Mona Lisa" .

wd:Q3211030 ex:something ex:Exhibition1 ;
    ex:somethingelse "Birth of Venus" .

ex:Exhibition1 a ex:Exhibition ;
    ex:title "The ancient Roman Empire Exhibition" .




# Join two Graphs

In [27]:
def jointwographs(graph1filename, graph2filename, finalgraphfilename):
  g1 = rdflib.Graph()
  g2 = rdflib.Graph()
  g1.parse(graph1filename, format="turtle")
  g2.parse(graph2filename, format="turtle")
  finalgraph = g1+g2
  print(graph.serialize(format="turtle").decode("utf-8"))
  graph.serialize(destination=finalgraphfilename, format="turtle")
